In [1]:
pip install praw

     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 204kB 25.8MB/s 


In [2]:
import praw
import pandas as pd
# Reddit credentials, password stored in .env 
# PRAW setup and login goes here, omitted for privacy

In [3]:
# set up some sort of corpus of keywords to snag specific reddit entries
all_locs = pd.read_json('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json')
all_locs = pd.json_normalize(all_locs['data'])
all_locs = all_locs.drop(columns=['edit_at','id'])
def cleanlinks(json):
    links_out = []
    for link in json:
        links_out.append(link['url'])
    return links_out
all_locs['links'] = all_locs['links'].apply(cleanlinks)
all_locs['date'] = pd.to_datetime(all_locs['date'],format='%Y-%m-%d')
all_tags = all_locs['tags'].copy()
tags = set()
for taglist in all_tags:
  for tag in taglist:
    if tag not in tags:
      tags.add(tag)
# manually remove needless tags
print(tags)
tags.discard('')
tags.discard('medic')
tags.discard('bike')
tags.discard('non-protest')
tags.discard('pregnant')
tags.discard('lgbtq+')
tags.discard('racial-profiling')
tags.discard('legal-observer')
tags.discard('tear-gas-canister')
tags.discard('politician')
tags.discard('incitement')
tags.discard('homeless')
tags.discard('shoot')
tags.discard('strike')
tags.discard('elderly')
tags.discard('vehicle')
tags.discard('inhumane-treatment')
tags.discard('journalist')
tags.discard('throw')
tags.discard('explosive')
tags.discard('threaten')
tags.discard('horse')
tags.discard('shove')
tags.discard('child')
tags.discard('shield')
tags.discard('dog')
tags.discard('knee')
tags.discard('protester')
tags.discard('gun')
tags.discard('conceal')
tags.discard('bystander')
tags.discard('grab')
tags.discard('push')
tags.discard('zip-tie')
tags.discard('spray')
tags.discard('drive')
tags.discard('person-with-disability')
tags.discard('celebrity')
tags.discard('projectile')
tags.discard('beat')
newtags = set()
for tag in tags:
  tag = newtags.add(tag.replace('-',' '))
tags = newtags
print(tags)

{'beat', '', 'arrest', 'wooden-bullet', 'throw', 'choke', 'pregnant', 'medic', 'mace', 'explosive', 'pepper-ball', 'horse', 'tear-gas-canister', 'child', 'knee-on-neck', 'conceal', 'legal-observer', 'spray', 'punch', 'pepper-spray', 'racial-profiling', 'protester', 'property-destruction', 'celebrity', 'rubber-bullet', 'abuse-of-power', 'push', 'gun', 'drive', 'elderly', 'non-protest', 'live-round', 'bean-bag', 'hide-badge', 'journalist', 'politician', 'incitement', 'projectile', 'gas', 'tear-gas', 'kick', 'shield', 'bystander', 'grab', 'baton', 'foam-bullet', 'lrad', 'taser', 'headlock', 'dog', 'strike', 'person-with-disability', 'threaten', 'homeless', 'body-cam', 'inhumane-treatment', 'shoot', 'tackle', 'shove', 'marking-round', 'lgbtq+', 'death', 'tase', 'vehicle', 'stun-grenade', 'bike', 'knee', 'zip-tie'}
{'arrest', 'choke', 'mace', 'knee on neck', 'wooden bullet', 'punch', 'foam bullet', 'tear gas', 'gas', 'rubber bullet', 'kick', 'abuse of power', 'baton', 'lrad', 'taser', 'head

In [4]:
# Grabbing 100 hottest posts on Reddit at the moment. Will filter for police use of force later

data = []

for submission in reddit.subreddit("news").hot(limit=100):
  data.append([submission.id, submission.title, submission.score, submission.subreddit, submission.url, 
               submission.num_comments, submission.selftext, submission.created])

# We'll need a way to get coordinates for a given post, before we include that in df
col_names = ['id', 'title', 'score', 'subreddit', 'url', 
             'num_comments', 'text', 'created']
df = pd.DataFrame(data, columns=col_names)

df.head()

,id,title,score,subreddit,url,num_comments,text,created
0,icq91v,Pennsylvania Among States To Sue Over Changes ...,6952,news,https://levittownnow.com/2020/08/18/pa-among-s...,223,,1.597881e+09
1,icm712,New Mexico sues over US Postal Service changes.,67206,news,https://www.kob.com/new-mexico-news/new-mexico...,2116,,1.597867e+09
2,icsg56,Mark Zuckerberg Is Despoiling a Tiny Coastal V...,2618,news,https://www.wweek.com/news/2020/08/19/mark-zuc...,224,,1.597888e+09
3,icncj5,Man Accused Of Assaulting Sesame Place Worker ...,3852,news,http://levittownnow.com/2020/08/19/man-accused...,373,,1.597871e+09
4,icjmim,The first marijuana vending machines now avail...,15840,news,https://growcola.com/marijuana-vending-machine...,880,,1.597854e+09


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
# get the url of the reddit post
for url in df['url']:
  # get the HTML from the url
  r = requests.get(url)
  soup = BeautifulSoup(r.text)
  # get tags from metadata for the site
  sitetags = set()
  for meta in soup.find_all('meta'):
    if meta is None:
      continue
    meta = str(meta)
    meta = meta.lower()
    meta = re.sub('[\W_]+',' ', meta)
    for tag in str(meta).split():
      # print(tag)
      sitetags.add(tag)
  tags_final = sitetags & tags
  if tags_final:
    # some matches found, print title and matched tags
    print(soup.title, tags_final)

<title>4 teens, including juvenile, arrested in connection with Raleigh girl’s murder outside Bojangles’ </title> {'death'}
<title>Police searching for suspect who sucker-punched elderly sidewalk sweeper – Boston News, Weather, Sports | WHDH 7News</title> {'punch'}
